In [1]:
from dotenv import load_dotenv

load_dotenv()

import os
import re
import itertools
import json

from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_community.vectorstores import Milvus
from pymilvus import Collection
from uuid import uuid4

from datetime import datetime
from langchain.schema.runnable import RunnableLambda, RunnableParallel

from langchain_community.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_teddynote.retrievers import KiwiBM25Retriever

In [2]:
import os
import re

def get_pdf_paths(directory):
    pdf_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jsonl'):
                full_path = os.path.join(root, file)
                pdf_paths.append(full_path)

    return pdf_paths

# 사용 예시
directory_path = "./parsed_pdf/result_v10/text"
result = get_pdf_paths(directory_path)

for resul in result:
    resul.replace('\\', '/')


In [3]:
result

['./parsed_pdf/result_v10/text\\KISWeekly제1090호(20240628)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1091호(20240705)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1092호(20240712)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1093호(20240719)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1094호(20240726)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1095호(20240802)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1096호(20240809)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1097호(20240816)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1098호(20240823)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1099호(20240830)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1100호(20240906)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1101호(20240913)_docs_text.jsonl',
 './parsed_pdf/result_v10/text\\KISWeekly제1103호(20240927)_docs_text.jsonl',
 './parsed_p

In [4]:
import json
from langchain.schema import Document
import re
a = ''
document = []
filepath = result[-3]
documents = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        
        # 각 줄을 JSON으로 파싱
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        # Document 객체 생성 및 리스트에 추가
        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        documents.append(doc)
document.append(documents)

In [5]:
import jsonlines

def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())

URI = 'http://127.0.0.1:19530/'

from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

In [6]:
document = [j for i in document for j in i]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splitted_docs = text_splitter.split_documents(document)
model_name = ['Salesforce/SFR-Embedding-2_R', 'Alibaba-NLP/gte-Qwen2-7B-instruct',
              'BAAI/bge-multilingual-gemma2', 'intfloat/e5-mistral-7b-instruct',
              'jinaai/jina-embeddings-v3', 'McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp-supervised',
              'intfloat/multilingual-e5-large-instruct', 'intfloat/multilingual-e5-large',
              'BAAI/bge-m3', 'jhgan/ko-sroberta-multitask', 'upskyy/bge-m3-korean',
              'nlpai-lab/KoE5']

for i in range(len(model_name)):
    embeddings = HuggingFaceEndpointEmbeddings(
        model=model_name[i],
        task='feature-extraction',

    )
    vectorstore_text = Milvus(
        embedding_function=embeddings,
        connection_args={'uri':URI},
        index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
        collection_name=model_name[i].replace('/', '_') + '_1000_100'
    )

    save_docs_to_jsonl(splitted_docs, os.path.join('./semantic_test/', model_name[i].replace('/', '_')+'_1000_100'+'.jsonl'))

    uuids = [str(uuid4()) for _ in range(len(splitted_docs))]

    vectorstore_text.add_documents(
        documents=splitted_docs,
        ids = uuids
    )

In [8]:
collection_name=['openai_large_test']

In [55]:
from datasets import Dataset
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    context_recall,
    context_precision,
    AnswerCorrectness,
    # answer_correctness,
    answer_similarity
)
import json

def convert_to_list(example):
    if isinstance(example["contexts"], list):  # 이미 리스트이면 그대로 반환
        contexts = example["contexts"]
    else:
        try:
            contexts = json.loads(example["contexts"])  # JSON 문자열이면 변환
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {example['contexts']} - {e}")
            contexts = []  # 예외 발생 시 빈 리스트로 대체
    return {"contexts": contexts}


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splitted_docs = text_splitter.split_documents(document)
model_name = [
    # 'intfloat/multilingual-e5-large',
    # 'BAAI/bge-m3',
    # 'jhgan/ko-sroberta-multitask',
    # 'upskyy/bge-m3-korean',
    # 'nlpai-lab/KoE5',
    # 'voyageai/voyage-multilingual-2',
    'intfloat/multilingual-e5-base',
    'intfloat/multilingual-e5-small',
    'cateto/longformer-ko-sroberta-multitask-23040'
]

for i in range(len(model_name)):
    embeddings = HuggingFaceEndpointEmbeddings(
        model=model_name[i],
        # model_kargs={'device':'cpu'},
        # encode_kwargs={'normalize_embeddings': True}
    )
    vectorstore_text = Milvus(
        embedding_function=embeddings,
        connection_args={'uri':URI},
        index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
        collection_name=model_name[i].replace('.', '_').replace('-', '_').replace('/', '_') + '_1000_100'
    )

    save_docs_to_jsonl(splitted_docs, os.path.join('./semantic_test/', model_name[i].replace('/', '_')+'_1000_100'+'.jsonl'))

    uuids = [str(uuid4()) for _ in range(len(splitted_docs))]

    vectorstore_text.add_documents(
        documents=splitted_docs,
        ids = uuids
    )

    result_ = []
    # vectorstore_text = Milvus(
    # embedding_function=embeddings,
    # connection_args={'uri':URI},
    # index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    # collection_name=i
    # )

    add_date_to_inputs = RunnableLambda(lambda inputs: {
    **inputs,
        "current_date": datetime.now().strftime("%Y-%m-%d")
    })

    splitted_doc = []
    filepath = os.path.join('./semantic_test', model_name[i].replace('/', '_') + '_1000_100'+'.jsonl')
    # with open(filepath, 'r', encoding='utf-8') as file:
    #     for line in file:
    #         if line.startswith('\n('):
    #             continue
    #         data = json.loads(line)

    #         doc = Document(
    #             page_content=data['page_content'],
    #             metadata=data['metadata']
    #         )
    #         splitted_doc.append(doc)

    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

    milvus_retriever = vectorstore_text.as_retriever(
        search_kwargs={'k':10}
    )

    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    df = pd.read_csv("./qa_ragas_250302.csv", index_col=False)
    df = df[['question', 'ground_truth']]
    df.ground_truth = df.apply(lambda x: x.ground_truth.split("'")[1].split("'")[0], axis=1)

    test_dataset = Dataset.from_pandas(df)

    batch_dataset = [question for question in test_dataset["question"]]
    context = milvus_retriever.batch(batch_dataset)

    if "contexts" in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(["contexts"]).add_column("contexts", [[j.page_content for j in i] for i in context])
    else:
        test_dataset = test_dataset.add_column("contexts", [[j.page_content for j in i] for i in context])

    prompt = PromptTemplate.from_template(
    '''You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer simply.
    Answer in Korean.

    #Question:
    {question}
    #Context:
    {context}

    #Answer:'''
    )

    chain = (
        RunnableParallel(
            context=milvus_retriever,
            question=RunnablePassthrough()
        )
        # | add_date_to_inputs
        | prompt
        | llm
        | StrOutputParser()
    )

    answer = chain.batch(batch_dataset)
    answer

    if "answer" in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
    else:
        test_dataset = test_dataset.add_column("answer", answer)

    test_dataset = test_dataset.map(convert_to_list)

    for _ in range(3):
        result = evaluate(
            dataset=test_dataset,
            metrics=[
                context_recall,
                context_precision,
                answer_similarity,
                AnswerCorrectness(weights=[0.75, 0.25]),
                # answer_correctness
            ],
        )
        result_.append(result)

    with open(filepath.replace('test', 'test/t10').replace('.jsonl', '.json'), "w", encoding='utf-8') as json_file:
        json.dump(result_, json_file, indent=4)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

DataTypeNotSupportException: <DataTypeNotSupportException: (code=1, message=Field dtype must be of DataType)>

In [ ]:
from datasets import Dataset
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    context_recall,
    context_precision,
    AnswerCorrectness,
    # answer_correctness,
    answer_similarity
)
import json

def convert_to_list(example):
    if isinstance(example["contexts"], list):  # 이미 리스트이면 그대로 반환
        contexts = example["contexts"]
    else:
        try:
            contexts = json.loads(example["contexts"])  # JSON 문자열이면 변환
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {example['contexts']} - {e}")
            contexts = []  # 예외 발생 시 빈 리스트로 대체
    return {"contexts": contexts}


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splitted_docs = text_splitter.split_documents(document)
model_name = [
    # 'intfloat/multilingual-e5-large',
    # 'BAAI/bge-m3',
    # 'jhgan/ko-sroberta-multitask',
    # 'upskyy/bge-m3-korean',
    # 'nlpai-lab/KoE5',
    # 'voyageai/voyage-multilingual-2',
    'intfloat/multilingual-e5-base',
    'intfloat/multilingual-e5-small',
    'cateto/longformer-ko-sroberta-multitask-23040'
]

for i in range(len(model_name)):
    embeddings=UpstageEmbedding
    vectorstore_text = Milvus(
        embedding_function=embeddings,
        connection_args={'uri':URI},
        index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
        collection_name=model_name[i].replace('.', '_').replace('-', '_').replace('/', '_') + '_1000_100'
    )

    save_docs_to_jsonl(splitted_docs, os.path.join('./semantic_test/', model_name[i].replace('/', '_')+'_1000_100'+'.jsonl'))

    uuids = [str(uuid4()) for _ in range(len(splitted_docs))]

    vectorstore_text.add_documents(
        documents=splitted_docs,
        ids = uuids
    )

    result_ = []
    # vectorstore_text = Milvus(
    # embedding_function=embeddings,
    # connection_args={'uri':URI},
    # index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    # collection_name=i
    # )

    add_date_to_inputs = RunnableLambda(lambda inputs: {
    **inputs,
        "current_date": datetime.now().strftime("%Y-%m-%d")
    })

    splitted_doc = []
    filepath = os.path.join('./semantic_test', model_name[i].replace('/', '_') + '_1000_100'+'.jsonl')
    # with open(filepath, 'r', encoding='utf-8') as file:
    #     for line in file:
    #         if line.startswith('\n('):
    #             continue
    #         data = json.loads(line)

    #         doc = Document(
    #             page_content=data['page_content'],
    #             metadata=data['metadata']
    #         )
    #         splitted_doc.append(doc)

    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

    milvus_retriever = vectorstore_text.as_retriever(
        search_kwargs={'k':10}
    )

    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    df = pd.read_csv("./qa_ragas_250302.csv", index_col=False)
    df = df[['question', 'ground_truth']]
    df.ground_truth = df.apply(lambda x: x.ground_truth.split("'")[1].split("'")[0], axis=1)

    test_dataset = Dataset.from_pandas(df)

    batch_dataset = [question for question in test_dataset["question"]]
    context = milvus_retriever.batch(batch_dataset)

    if "contexts" in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(["contexts"]).add_column("contexts", [[j.page_content for j in i] for i in context])
    else:
        test_dataset = test_dataset.add_column("contexts", [[j.page_content for j in i] for i in context])

    prompt = PromptTemplate.from_template(
    '''You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer simply.
    Answer in Korean.

    #Question:
    {question}
    #Context:
    {context}

    #Answer:'''
    )

    chain = (
        RunnableParallel(
            context=milvus_retriever,
            question=RunnablePassthrough()
        )
        # | add_date_to_inputs
        | prompt
        | llm
        | StrOutputParser()
    )

    answer = chain.batch(batch_dataset)
    answer

    if "answer" in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
    else:
        test_dataset = test_dataset.add_column("answer", answer)

    test_dataset = test_dataset.map(convert_to_list)

    for _ in range(3):
        result = evaluate(
            dataset=test_dataset,
            metrics=[
                context_recall,
                context_precision,
                answer_similarity,
                AnswerCorrectness(weights=[0.75, 0.25]),
                # answer_correctness
            ],
        )
        result_.append(result)

    with open(filepath.replace('test', 'test/t10').replace('.jsonl', '.json'), "w", encoding='utf-8') as json_file:
        json.dump(result_, json_file, indent=4)